# simran4@wisc.edu, rgundavarapu@wisc.edu

In [1]:
from google.cloud import bigquery
bq = bigquery.Client(project="cs544-sp23-376003")
%load_ext google.cloud.bigquery

# Part 1

In [2]:
#q1
q1 = bq.query("""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
LIMIT 5
""")
df = q1.to_dataframe()
(df['geo_id'][0])

'55025'

In [3]:
#q2
q2 = bq.query("""
SELECT COUNT(DISTINCT(county_name)) AS MEOW,state_fips_code
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY MEOW DESC
LIMIT 5
""")
df = q2.to_dataframe()
q2_dict={}
for i in df.iterrows():
    q2_dict[i[1].state_fips_code]=i[1].MEOW
q2_dict

{'48': 254, '13': 159, '51': 129, '21': 120, '29': 114}

In [4]:
#q3
no_cache = bigquery.QueryJobConfig(use_query_cache=False)
q1 = bq.query("""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
LIMIT 5
""",job_config=no_cache)
q2 = bq.query("""
SELECT COUNT(DISTINCT(county_name)) AS MEOW,state_fips_code
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY MEOW DESC
LIMIT 5
""",job_config=no_cache)
q1cost=5 * (q1.total_bytes_billed / 1024**4)
q2cost=5 * (q2.total_bytes_billed / 1024**4)
q3_dict={}
q3_dict['q1']=q1cost
q3_dict['q2']=q2cost
q3_dict

{'q1': 4.76837158203125e-05, 'q2': 4.76837158203125e-05}

In [5]:
!python3 autograder.py p7.ipynb

Reading Question 1
Reading Question 2
Reading Question 3
Wrote results to p7.csv.
There were 2 errors:

Question 2: expected type to be dict, but found NoneType

Question 3: expected type to be dict, but found NoneType

7 answers not found, for question(s): 4, 5, 6, 7, 8, 9, 10

Result: 1 of 10 passed, for an estimated score of 10% (prior to grader deductions).
